In [ ]:
from numpy import zeros, ones, expand_dims, hstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets import cifar10
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, BatchNormalization, Activation
from matplotlib import pyplot
 

In [2]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## define discriminator model
there is 3 Conv layers followed by 1 dense layer to determine fake or realness <br>
to Create Q_model to calculate regularization term of new latent code we added 1 new 128 dimention Dense and 1 n_cat (information we are going to learn for images) dimentinal Dense layer with softmax


In [ ]:
# define the standalone discriminator model
def define_discriminator(n_cat, in_shape=(32,32,3)):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image input
  in_image = Input(shape=in_shape)
  # downsample to 16x16
  d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
  d = LeakyReLU(alpha=0.1)(d)
  print(d.shape)
  # downsample to 8x8
  d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
  d = LeakyReLU(alpha=0.1)(d)
  d = BatchNormalization()(d)
  # normal
  d = Conv2D(256, (4,4), padding='same', kernel_initializer=init)(d)
  d = LeakyReLU(alpha=0.1)(d)
  d = BatchNormalization()(d)
  # flatten feature maps
  d = Flatten()(d)
  # real/fake output
  out_classifier = Dense(1, activation='sigmoid')(d)
  # define d model
  d_model = Model(in_image, out_classifier)
  # compile d model
  d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
  # create q model layers
  q = Dense(128)(d)
  q = BatchNormalization()(q)
  q = LeakyReLU(alpha=0.1)(q)
  # q model output
  out_codes = Dense(n_cat, activation='softmax')(q)
  # define q model
  q_model = Model(in_image, out_codes)
  return d_model, q_model

## Define Generator
Relatively to discriminator we have 1 Dense layer followed by 1 Conv layer and used 2 Transpose Conv layer to up sample and to create new images from 2 latent input (noise and code)

In [ ]:
# define the standalone generator model
def define_generator(gen_input_size):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image generator input
  in_lat = Input(shape=(gen_input_size,))
  # foundation for 8x8 image
  n_nodes = 512 * 8 * 8
  gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
  gen = Activation('relu')(gen)
  gen = BatchNormalization()(gen)
  gen = Reshape((8, 8, 512))(gen)
  # normal
  gen = Conv2D(128, (4,4), padding='same', kernel_initializer=init)(gen)
  gen = Activation('relu')(gen)
  gen = BatchNormalization()(gen)
  # upsample to 16x16
  gen = Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  gen = Activation('relu')(gen)
  gen = BatchNormalization()(gen)
  # upsample to 32x32
  gen = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(gen)
  # tanh output
  out_layer = Activation('tanh')(gen)
  # define model
  model = Model(in_lat, out_layer)
  return model

## Define InfoGAN
here we create discriminator q and generator model and compile them with Adam optimizer and 0.0002 learning rate

In [ ]:
# define the combined discriminator, generator and q network model
def define_gan(g_model, d_model, q_model):
  # make weights in the discriminator (some shared with the q model) as not trainable
  d_model.trainable = False
  # connect g outputs to d inputs
  d_output = d_model(g_model.output)
  # connect g outputs to q inputs
  q_output = q_model(g_model.output)
  # define composite model
  model = Model(g_model.input, [d_output, q_output])
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=opt)
  print(model.summary)
  return model

## Load Cifar10 Data set and sacel it to [-1,1]

In [ ]:
# load images
def load_real_samples():
	# load dataset
	(trainX, _), (_, _) = cifar10.load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	print(X.shape)
	return X

## Generate real images 
generate n_sample from dataset which are real images

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images and labels
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

## Generate latent points
Generate latent point from 2 inputs spaces (noise and code)<br>
then these latent poing will be given to Generator model to produce new images

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_cat, n_samples):
	# generate points in the latent space
	z_latent = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_latent = z_latent.reshape(n_samples, latent_dim)
	# generate categorical codes
	cat_codes = randint(0, n_cat, n_samples)
	# one hot encode
	cat_codes = to_categorical(cat_codes, num_classes=n_cat)
	# concatenate latent points and control codes
	z_input = hstack((z_latent, cat_codes))
	return [z_input, cat_codes]

## Fake sample generator
Generator then predict from these latent points and generate new samples that are fake and differs from real ones

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_cat, n_samples):
	# generate points in latent space and control codes
	z_input, _ = generate_latent_points(latent_dim, n_cat, n_samples)
	# predict outputs
	images = generator.predict(z_input)
	# create class labels
	y = zeros((n_samples, 1))
	return images, y

## Plot and save
we save our images and models every epoch in case of disconnectivity

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, gan_model, latent_dim, n_cat, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_cat, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0])
	# save plot to file
	filename1 = 'images/%d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = 'model/%d.h5' % (step+1)
	g_model.save(filename2)
	# save the gan model
	filename3 = 'gan_model/%d.h5' % (step+1)
	gan_model.save(filename3)
	print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

## Train
here for each epoch and for each image in batch size which is 64 we generate real and fake images and predict their validity with discriminator<br>
we take points from latent spaces and feed them to generator and made it to produce new images then we compute its loss and update weights to lear our network <br>
after some epoches we get better images

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_cat, n_epochs=20, n_batch=64):

  d1_loss_history = []
  d2_loss_history = []
  g1_loss_history = []
  g2_loss_history = []

  # calculate the number of batches per training epoch
  bat_per_epo = int(dataset.shape[0] / n_batch)
  # calculate the number of training iterations
  n_steps = bat_per_epo * n_epochs
  # calculate the size of half a batch of samples
  half_batch = int(n_batch / 2)
  # manually enumerate epochs
  for i in range(n_steps):
    # get randomly selected 'real' samples
    X_real, y_real = generate_real_samples(dataset, half_batch)
    # update discriminator and q model weights
    d_loss1 = d_model.train_on_batch(X_real, y_real)
    # generate 'fake' examples
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_cat, half_batch)
    # update discriminator model weights
    d_loss2 = d_model.train_on_batch(X_fake, y_fake)
    # prepare points in latent space as input for the generator
    z_input, cat_codes = generate_latent_points(latent_dim, n_cat, n_batch)
    # create inverted labels for the fake samples
    y_gan = ones((n_batch, 1))
    # update the g via the d and q error
    _,g_1,g_2 = gan_model.train_on_batch(z_input, [y_gan, cat_codes])
    # summarize loss on this batch
    print('>%d, d[%.3f,%.3f], g[%.3f] q[%.3f]' % (i+1, d_loss1, d_loss2, g_1, g_2))
    d1_loss_history.append(d_loss1)
    d2_loss_history.append(d_loss2)
    g1_loss_history.append(g_1)
    g2_loss_history.append(g_2)
    # evaluate the model performance every 'epoch'
    if (i+1) % (bat_per_epo * 10) == 0:
      summarize_performance(i, g_model, gan_model, latent_dim, n_cat)
  return d1_loss_history, d2_loss_history, g1_loss_history, g2_loss_history

In [ ]:
# number of values for the categorical control code
n_cat = 10
# size of the latent space
latent_dim = 62
# create the discriminator
d_model, q_model = define_discriminator(n_cat)
# create the generator
gen_input_size = latent_dim + n_cat
g_model = define_generator(gen_input_size)
# create the gan
gan_model = define_gan(g_model, d_model, q_model)
# load image data
dataset = load_real_samples()
# train model
d1_loss_history, d2_loss_history, g1_loss_history, g2_loss_history = train(g_model, d_model, gan_model, dataset, latent_dim, n_cat)

(None, 16, 16, 64)
<bound method Model.summary of <tensorflow.python.keras.engine.functional.Functional object at 0x7fea33223128>>
170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3, 1)
>1, d[1.192,0.736], g[0.692] q[2.319]
>2, d[0.400,0.490], g[0.693] q[2.324]
>3, d[0.070,0.318], g[0.695] q[2.253]
>4, d[0.076,0.226], g[0.697] q[2.263]
>5, d[0.060,0.124], g[0.699] q[2.292]
>6, d[0.035,0.097], g[0.700] q[2.210]
>7, d[0.029,0.075], g[0.701] q[2.266]


KeyboardInterrupt: ignored

## Plot loss functions

In [ ]:
def plot_parameter(input, title)
  pyplot.plot(input)
  pyplot.title(title)
  pyplot.xlabel('Epoch')
  pyplot.ylabel('loss')
  pyplot.show()
plot_parameter(d1_loss_history, "d1_loss")
plot_parameter(d2_loss_history, 'd2_loss')
plot_parameter(g1_loss_history, 'g1_loss')
plot_parameter(g2_loss_history, 'g2_loss')